In [3]:
from urllib.request import urlopen
from io import TextIOWrapper

msd_db="modelseed_biochemistry"
spark = get_spark_session()
create_namespace_if_not_exists(spark, msd_db)

header = True
structures_url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/refs/heads/dev/Biochemistry/Structures/Unique_ModelSEED_Structures.txt"
structures_dict = dict()
with urlopen(structures_url) as response:
	lines = TextIOWrapper(response)
	for line in lines:
		if(header is True):
			header = False
			continue

		line=line.strip('\r\n')
		tmp_lst = line.split('\t')
		(cpd,struct_type,struct)=(tmp_lst[0],tmp_lst[1],tmp_lst[-1])

		# fixing typo
		if(struct_type == 'SMILE'):
			struct_type+='S'

		if(cpd not in structures_dict):
			structures_dict[cpd]={'compound_id':cpd}
		structures_dict[cpd][struct_type.lower()]=struct

print(structures_dict['cpd00001'])

structures = list()
for cpd in structures_dict:

    # update identifier
    structures_dict[cpd]['compound_id'] = 'seed.compound:'+structures_dict[cpd]['compound_id']
    
    structures.append(structures_dict[cpd])

rdd = spark.sparkContext.parallelize(structures)
structs_df = spark.read.json(rdd)

spark.sql("USE modelseed_biochemistry")
spark.sql("DROP TABLE IF EXISTS molecular_structure")

25/01/09 23:27:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Namespace modelseed_biochemistry is ready to use.
{'compound_id': 'cpd00001', 'smiles': 'O', 'inchikey': 'XLYOFNOQVPJJNP-UHFFFAOYSA-N', 'inchi': 'InChI=1S/H2O/h1H2'}


25/01/09 23:27:17 WARN TaskSetManager: Stage 0 contains a task of very large size (5430 KiB). The maximum recommended task size is 1000 KiB.
25/01/09 23:27:21 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [5]:
from pyspark.sql.types import (
	StringType, LongType, DoubleType, BooleanType
)
from pyspark.sql.functions import col

for field, dtype in structs_df.dtypes:
        
    if(dtype == 'string'):
        structs_df = structs_df.withColumn(field, col(field).cast(StringType()))
    elif(dtype == 'double'):
        structs_df = structs_df.withColumn(field, col(field).cast(DoubleType()))
    elif(dtype == 'bigint' and field == 'is_transport'):
        structs_df = structs_df.withColumn(field, col(field).cast(BooleanType()))
    elif(dtype == 'bigint'):
        structs_df = structs_df.withColumn(field, col(field).cast(LongType()))
    else:
        print("Unsupported field?",field,dtype)

print("Finished updating dtypes")

Finished updating dtypes


In [6]:
spark_table = f"{msd_db}.molecular_structure"
delta_file = f"msd_delta/molecular_structure.delta"
structs_df.write.mode("overwrite") \
        .option("overwriteSchema", "true") \
    	.option("compression", "snappy") \
    	.option("path", f"s3a://cdm-lake/{delta_file}") \
    	.format("delta") \
    	.saveAsTable(spark_table)
print(f"Spark table {spark_table} created.")

25/01/09 23:28:09 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/01/09 23:29:10 WARN TaskSetManager: Stage 8 contains a task of very large size (5430 KiB). The maximum recommended task size is 1000 KiB.
25/01/09 23:30:01 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`modelseed_biochemistry`.`molecular_structure` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/01/09 23:30:01 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/01/09 23:30:01 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/01/09 23:30:01 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/01/09 23:30

Spark table modelseed_biochemistry.molecular_structure created.
